# LangChain + Web Data
**2024 삼성전자 무선사업부 생성형 AI 교육**

1. 기본 패키지 설정
2. 소스 링크 수집
3. 데이터 불러오기
4. 데이터 Split
5. 데이터 Embedding
6. LLM 생성
7. Chain 연결

In [ ]:
# 1. 패키지 설치

%pip install chromadb
%pip install unstructured

In [ ]:
# 2. 기본 패키지 설정 & AzureOpenAI 환경설정

# 환경 변수 설정!! 
import os
import langchain

from config_azure import (
    AZURE_OPENAI_API_VERSION,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY
)
# lagchain 사용시 반드시 환경변수 값 사용!!
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

In [ ]:
# 3. 소스 링크 수집
# [링크 바로가기](https://www.xn--hq1bn9iz0nvzar4a.net/%ec%95%a0%ec%9b%94-%ed%98%91%ec%9e%ac-%eb%a7%9b%ec%a7%91-best-10/")

urls = [
    "https://www.xn--hq1bn9iz0nvzar4a.net/%ec%95%a0%ec%9b%94-%ed%98%91%ec%9e%ac-%eb%a7%9b%ec%a7%91-best-10/",
    "https://www.xn--hq1bn9iz0nvzar4a.net/%ec%95%a0%ec%9b%94-%eb%a7%9b%ec%a7%91-best-10/",
    "https://m.blog.naver.com/miya75kr/223390881363",
    "https://m.blog.naver.com/kongmistar/223444031844",
    "https://m.blog.naver.com/psmr1234/223188197427",
    "https://www.xn--hq1bn9iz0nvzar4a.net/%ec%a0%9c%ec%a3%bc-%ea%b3%b5%ed%95%ad-%ea%b7%bc%ec%b2%98-%eb%a7%9b%ec%a7%91-best-10/",
    "https://www.xn--hq1bn9iz0nvzar4a.net/%ec%a0%9c%ec%a3%bc%eb%8f%84-%eb%a8%b9%ea%b1%b0%eb%a6%ac-best-10/"
]


In [ ]:
# 4. URL loader 생성-데이터 읽어오기
from langchain.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()
data

In [ ]:
# 6. Text Split(GPT API Token Limit)

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap=100,
                                               separators=["\n\n", "\n", "(?<=\. )", " ", ""],)
doc = text_splitter.split_documents(data)
doc

### 참고 링크
* [Understanding LangChain's RecursiveCharacterTextSplitter](https://dev.to/eteimz/understanding-langchains-recursivecharactertextsplitter-2846)  
* [Chunkerizer 테스터기](https://chunkerizer.streamlit.app)

In [ ]:
# 6. Vetor Store에 텍스트 벡터값 저장(검색가능함)

from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma

# 임베딩 모델 생성
embedding_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
)

# 문서 -> 벡터 변환
vectorstore = Chroma.from_documents(doc, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.33}
# )

In [ ]:
vectorstore.similarity_search_with_score("두부")

In [ ]:
# 7. OpenAI LLM(Chat Model 연결)

from langchain_openai import AzureChatOpenAI

chat_llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    max_tokens=512,
    temperature=0.2,
)

In [ ]:
# 8. Chain 연결 without LCEL

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False)

In [ ]:
# 9. 질의
query1 = {"query" : "갈치 식당 소개해줘"}
query2 = {"query" : "두부 요리 파는 곳 있어?"}
query3 = {"query" : "제갈양 식당 유명해?"}

queries = [query1, query2, query3]


from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    result = qa_chain.batch(queries)
    print(result)
    print(cb)

In [ ]:
# 10. Custom Prompt

from langchain.prompts import PromptTemplate

custom_template = '''아래 자료를 기반으로 사용자 질문에 답변한다.
자료에 내용이 없는 경우 모른다고하고, 내용을 지어내지 않는다.

* 자료 : {context}
* 사용자 질문 : {question}
'''
custom_prompt = PromptTemplate(
    template=custom_template,
    input_variables=[
        'context',
        'question',
    ]
)

new_chain = RetrievalQA.from_chain_type(llm=chat_llm,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        return_source_documents=False,
                                        chain_type_kwargs={"prompt": custom_prompt})

new_result = new_chain.batch(queries)
print(new_result)

In [ ]:
# 11. Chain 연결(LCEL)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 검색된 문서를 string으로 변환 
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

lcel_chain = (
        {
        "context": retriever | format_docs,
        "question": RunnablePassthrough() # RunnablePassthrough : 데이터를 그대로 전달
    }
    | custom_prompt
    | chat_llm
    | StrOutputParser()
)

lcel_chain.invoke("서울 맛집 알려줘")

In [ ]:
# 여러개 질문 한번에 하기

lcel_queries = ["갈치 식당 소개해줘", "두부 요리 파는 곳 있어?", "제갈양 식당 유명해?"]

lcel_chain.batch(lcel_queries)